In [1]:
import sys
sys.path.append('../analysis_v2/')
import SiemensQuadraProperties as sqp
import CoincidenceGeneration as cg
import SimulationDataset as sd
import PhysicsConstants as pc
import GATEfileWriter as gf

import time

import matplotlib.pyplot as plt
params = {'legend.fontsize': 15,
          'legend.title_fontsize': 15,
          'legend.loc': "upper left",
          'axes.labelsize': 15,
          'xtick.labelsize': 15,
          'ytick.labelsize': 15}
plt.rcParams.update(params)

# Fix random seed for reproducibility, or leave blank to allow the results to vary
import numpy as np
RNG = np.random.default_rng(1234)

# Set batch size for coincidence generation
# Larger batches will use more RAM, and will eventually become inefficient due to wasted events
# Smaller batches are inefficient due to overheads
# 256-1024 seems to be about right
BATCH_SIZE=1024

Using an analogous structure to the NECR notebook, but this just creates a single data sample, and then saves it in a GATE-compatible ROOT file

In [2]:
# Simulation parameters
detectorLength = 1024
phantomLength = 700
datasetSize = 1000000
siemensEmin = 435.0
siemensEmax = 585.0
detectorMaterial = "LSO"
timeSec = 0
fileName = "testReconstructionData"

start = time.time_ns()

# Make data samples
# Have to use the full-detail (Crystal) version of the geometry to get the GATE module IDs
# 1) High activity offset
tracer1Data = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "LinearF18", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True,
                               SourceOffset=30, STIRheader=fileName+".hroot" )
tracer1Activity = pc.TracerActivityAtTime( 1100E6, timeSec, "F18" )

# 2) low activity central
tracer2Data = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "LinearF18", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True,
                               SourceOffset=0 )
tracer2Activity = pc.TracerActivityAtTime( 400E6, timeSec, "F18" )

# 3) Intrinsic background
crystalData = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "Siemens", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True )
crystalActivity = sqp.Lu176decaysInMass( sqp.DetectorMass(detectorMaterial) )

# Create a moduleID lookup table
moduleIDs = tracer1Data.GetModuleIDs()
moduleIDs.update( tracer2Data.GetModuleIDs() )
moduleIDs.update( crystalData.GetModuleIDs() )

end = time.time_ns()
print( "Photon simulation (if needed) and data loading: " + str( (end-start)/1e9 ) + "s" )
start = time.time_ns()

# Assemble data sources, including crystal intrinsic background
activityList = [ tracer1Activity, tracer2Activity, crystalActivity ]
dataList = [ tracer1Data, tracer2Data, crystalData ]

# Create a generator object that will simulate radioactive decays and load Geant4 events
generator = cg.GenerateCoincidences( BATCH_SIZE, activityList, dataList, RNG, CoincidenceWindow=4.7, SimulationWindow=1E7, \
                                     MultiWindow=False, EnergyResolution=0.0, EnergyMin=0.0, EnergyMax=0.0, TimeResolution=0.0 )

# Save photons to a GATE format file
gf.GATEfromGenerator( BATCH_SIZE, generator, fileName+".root", moduleIDs, sqp.DetectorRadius(), PairMode="Exclusive", ZMin=-325.0, ZMax=325 )

end = time.time_ns()
print( "Coincidence generation and writing: " + str( (end-start)/1e9 ) + "s" )

Creating dataset hits.n1000000.SiemensCrystal.1024mm.LinearF18.700mm.-y30mm.1234.csv
Running command =  ../build/SimplePetScanner -n 1000000 --detector SiemensCrystal --detectorLengthMM 1024 --source LinearF18 --phantomLengthMM 700 --outputFileName hits.n1000000.SiemensCrystal.1024mm.LinearF18.700mm.-y30mm.1234.csv --randomSeed 1234 --sourceOffsetMM 30 --detectorMaterial LSO --STIRheader testReconstructionData.hroot

**************************************************************
 Geant4 version Name: geant4-11-03 [MT]   (6-December-2024)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************

G4PhysListFactory::GetReferencePhysList <QGSP_BIC_HP_EMY>  EMoption= 3
<<< Geant4 Physics List si

ERROR: Can not open a macro file <run.mac>. Set macro path with "/control/macroPath" if needed.



-------- WWWW ------- G4Exception-START -------- WWWW -------
*** G4Exception : UIMAN0123
      issued by : G4UImanager::ApplyCommand
Command aborted (400)
Error code : 400
*** This is just a warning message. ***
-------- WWWW -------- G4Exception-END --------- WWWW -------

======                 Electromagnetic Physics Parameters      ========
LPM effect enabled                                 1
Enable creation and use of sampling tables         0
Apply cuts on all EM processes                     0
Use combined TransportationWithMsc                 Disabled
Use general process                                1
Enable linear polarisation for gamma               0
Enable photoeffect sampling below K-shell          1
Enable sampling of quantum entanglement            0
X-section factor for integral approach             0.8
Min kinetic energy for tables                      10 eV 
Max kinetic energy for tables                      100 TeV
Number of bins per decade of a table            

In [3]:
import STIRreconstruction as sr

sinogramName = "testSino"
sr.CreateSinogram( fileName, sinogramName )

Number of rings
Number of detectors per ring
Inner ring diameter (cm)
Average depth of interaction (cm)
Distance between rings (cm)
Default bin size (cm)
View offset (degrees)
Maximum number of non-arc-corrected bins
Default number of arc-corrected bins
number of submodules_Y
number of submodules_Z
number of crystals_Z
number of crystals_Y
Number of rings
number of crystals_Y
BEN BEN BEN I'm reading the header!
BEN BEN BEN I think the root file is 
BEN BEN BEN I'm reading a root file: testReconstructionData.root
BEN BEN BEN this->chain_name.c_str(): Coincidences


BEN BEN BEN read_optional_root_fields: 0

INFO: InputStreamFromROOTFile: Processing data with the following event type inclusions:
  Unscattered from same eventID:        1
  Unscattered from different eventIDs:  0
  Scattered from same eventID:          0
  Scattered different eventIDs:         0
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::set_up
BEN BEN BEN error here in scanner
BEN BEN BEN initialise_max_FOV_radius()
BEN BEN BEN initialise_max_FOV_radius() done
BEN BEN BEN leaving scanner maybe not
BEN BEN BEN actually this
BEN BEN BEN actually not

INFO: Tbin -137: -366.871 - -364.20285 mm (-2447.5 - -2429.7 ps) = 2.6681519

INFO: Tbin -136: -364.20285 - -361.5347 mm (-2429.7 - -2411.9 ps) = 2.6681519

INFO: Tbin -135: -361.5347 - -358.86655 mm (-2411.9 - -2394.0999 ps) = 2.6681519

INFO: Tbin -134: -358.86655 - -356.1984 mm (-2394.0999 - -2376.2998 ps) = 2.6681519

INFO: Tbin -133: -356.1984 - -353.53024 mm (-2376.2998 - -2358.5 ps) = 2.6681519

INFO: Tbin -132: -353.530






































































































































































































































































































































LmToProjData NOT Using FRAME_BASED_DT_CORR
lm to projdata parameters :=
input file := testReconstructionData.hroot
template projdata := testReconstructionData.hs
frame definition file := 
num events to store := -1
output filename prefix := ./testSino
bin normalisation type for pre-normalisation := None

bin normalisation type for post-normalisation := None

maximum absolute segment number to process := 5
do pre normalisation := 0
num tof bins in memory := 1
num segments in memory := 11
store prompts := 1
store delayeds := 1
list event coordinates := 0
end := 















































































































BEN BEN BEN by view 0, 4
BEN BEN BEN by view 0, 5
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::get_next_record
BEN BEN BEN InputStreamFromROOTFile::check_brentry_randoms_scatter_energy_conditions FAIL random
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::get_next_record continue due to failed check
BEN BEN BEN InputStreamFromROOTFile::check_brentry_randoms_scatter_energy_conditions FAIL random
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::get_next_record continue due to failed check
BEN BEN BEN InputStreamFromROOTFile::check_brentry_randoms_scatter_energy_conditions FAIL energy
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::get_next_record continue due to failed check
BEN BEN BEN InputStreamFromROOTFile::check_brentry_randoms_scatter_energy_conditions FAIL random
BEN BEN BEN InputStreamFromROOTFileForCylindricalPET::get_next_record continue due to failed check
BEN BEN BEN InputStreamFromROOTFile::check_brentry_randoms_scatter_energy_conditions FAIL random


Number of prompts stored in this time period : 0
Number of delayeds stored in this time period: 0
Last stored event was recorded before time-tick at 0 secs
Early stop due to EOF. 
Total number of counts (either prompts/trues/delayeds) stored: 0

This took 0.06s CPU time.
